In [1]:
experiment_name = '25similar'
%load_ext autoreload
%autoreload 2
%matplotlib inline
%run init.py

Setting up variables & DB connection for experiment:

*******************
_exp3
*******************

Snorkel session connected to:  postgres:///snorkel_exp3


/home/antonis/anaconda2/envs/snorkel27clone/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import seaborn as sns
from snorkel.annotations import save_marginals

In [3]:
from snorkel.lf_helpers import *
import pickle,glob

/home/antonis/anaconda2/envs/snorkel27clone/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
import pandas as pd
from MLutils import cohen_kappa_score, plot_marginals_histogram, neg_to_bin_labels

In [5]:
from MLutils import diversity_heatmap, merge_pickles_pred_dicts

In [6]:
# TODO move in LSTM training section

# needed to train the LSTM here
train = session.query(REGULATOR).filter(REGULATOR.split == 0).order_by(REGULATOR.id).all()
dev = session.query(REGULATOR).filter(REGULATOR.split == 1).order_by(REGULATOR.id).all()
test = session.query(REGULATOR).filter(REGULATOR.split == 2).order_by(REGULATOR.id).all()
unlab = session.query(REGULATOR).filter(REGULATOR.split == 3).order_by(REGULATOR.id).all()

In [7]:
from snorkel.annotations import LabelAnnotator

In [8]:
def signal_annotator(c):
    """A generator over the different (worker_id, label_id) pairs for a Tweet."""
    for model in results_dict.keys():
        try:
            yield model, results_dict[model]['label_unlab'][c.id]
            
        except:
            try:
                yield model, results_dict[model]['label_val'][c.id]
            except:
                yield model, results_dict[model]['label_test'][c.id]

labeler = LabelAnnotator(label_generator=signal_annotator)

# Retrieve results and pick models

In [ ]:
import glob
from MLutils import diversity_matrix, reduce_results_dict, balance_candidates, sample_from_csr, majority_vote, majority_vote_score, average_vote, error_analysis
from utils import check_class_imbalance
from itertools import combinations
from snorkel.learning import RandomSearch, GenerativeModel, GridSearch
from snorkel.learning.structure import DependencySelector
from snorkel.learning import reRNN

from sklearn.metrics import accuracy_score, cohen_kappa_score

In [10]:
l = glob.glob(path_base_learners+"/KMeansPeaks/*.pickle")
l.sort()
l

['pickles/_exp3/base_learner_predictions/KMeansPeaks/10clusters.pickle',
 'pickles/_exp3/base_learner_predictions/KMeansPeaks/13clusters.pickle',
 'pickles/_exp3/base_learner_predictions/KMeansPeaks/15clusters.pickle',
 'pickles/_exp3/base_learner_predictions/KMeansPeaks/20clusters.pickle',
 'pickles/_exp3/base_learner_predictions/KMeansPeaks/21clusters.pickle',
 'pickles/_exp3/base_learner_predictions/KMeansPeaks/5clusters.pickle']

In [87]:
test_scores = pd.DataFrame(index=l)
test_scores['n_classif'] = list(map(lambda x: x.split('/')[-1] ,test_scores.index))

In [12]:
result_dict_file = l[0]

In [102]:
vec = gen_model.score(L_test, L_gold_test)

In [104]:
vec

(0.5658274865868758, 0.6910282258064516, 0.6221919673247107)

In [13]:
for result_dict_file in l[1:]:
    with open(result_dict_file, 'rb') as f:
        results_dict = pickle.load(f)
    print 'Loaded:\n',
    print results_dict.keys()

    selected_models = results_dict.keys()
    selected_models.sort()

    # Load predicted labels from classifiers


    # convert labels to -1,1....
    for model in selected_models:
        results_dict[model]['label_unlab'] = dict(zip(results_dict[model]['label_unlab_prob+'].keys(),(np.array(results_dict[model]['label_unlab_prob+'].values())>=0.5)*2-1))
        results_dict[model]['label_val'] = dict(zip(results_dict[model]['label_val_prob+'].keys(),(np.array(results_dict[model]['label_val_prob+'].values())>=0.5)*2-1))
        results_dict[model]['label_test'] = dict(zip(results_dict[model]['label_test_prob+'].keys(),(np.array(results_dict[model]['label_test_prob+'].values())>=0.5)*2-1))


    ## Port results into snorkel

    ### Apply labeler / load label matrix from pickle

    #regenerate L_unlab - Takes about 1h for 12 voters (100% coverage)

    L_unlab = labeler.apply(split=3, parallelism=6)

    # load validation & test set & teir labels
    L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
    L_dev = labeler.apply_existing(split=1, parallelism=6)

    L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
    L_test = labeler.apply_existing(split=2, parallelism=6)

    # load unlabeled set labels (for exp3)
    L_gold_unlab = load_gold_labels(session, annotator_name='gold', split=3)

    ### Check some label statistics



    #** Majority voting **
    error_analysis(L_dev, L_gold_dev, majority_voting=True, save_name=result_dict_file+'MV.png')
    test_scores.loc[result_dict_file, 'MV_P'] , test_scores.loc[result_dict_file, 'MV_R'] , test_scores.loc[result_dict_file, 'MV_F1']  = majority_vote_score(L_dev, L_gold_dev)


    #** Average voting **
    error_analysis(L_dev, L_gold_dev, average_voting=True,  save_name=result_dict_file+'AvgV.png')
    
    
    # Denoising part

    ## Find dependencies between LFs
    ds = DependencySelector()

    # Generate all possible dependencies with threshold from 0 to 0.15
    all_deps = [ds.select(L_unlab, threshold=thresh) for thresh in np.array(range(0.1,15.1,2.5))/100.]
    print list(map(lambda x: len(x) ,all_deps))
    all_deps = list(np.unique(all_deps)) # delete duplicates for gridsearch
    print list(map(lambda x: len(x) ,all_deps))

    print "Maximum nr of dependencies: %i" %len(list(combinations(L_dev.col_index.values(),2)))

    # GM training



    # without HyperParam search
    gen_model = GenerativeModel()
    gen_model.train(L_unlab)
    test_scores.loc[result_dict_file, 'GM_default_P'], test_scores.loc[result_dict_file, 'GM_default_R'], test_scores.loc[result_dict_file, 'GM_default_F1'] = majority_vote_score(L_test, L_gold_test)


    #** Hyperparameter search **

    param_ranges = { # parameters for the train function of the GenerativeModel
        'step_size' : [10./L_unlab.shape[0],
                      5./L_unlab.shape[0],
                       15./L_unlab.shape[0]
                      ],
        'decay' : [0.99],
        'epochs' : [50],
        'reg_param': [1e-1,1e-2,1e-3],
        'deps': all_deps
    }

    model_class_params = {  # parameters for the GenerativeModel
        'lf_propensity': [True]
    }

    searcher = RandomSearch(GenerativeModel, param_ranges,  L_unlab, model_class_params = model_class_params , n=24 )

    gen_model, run_stats = searcher.fit(L_dev, L_gold_dev, n_threads=6 )
    
    test_scores.loc[result_dict_file, 'GM_hyperp_P'], test_scores.loc[result_dict_file, 'GM_hyperp_R'], test_scores.loc[result_dict_file, 'GM_hyperp_F1'] = gen_model.score(L_test, L_gold_test)

    # Save marginals and proceed to LSTM training

    unlab_marginals_gen = gen_model.marginals(L_unlab)
    unlab_marginals_avg = average_vote(L_unlab)
    unlab_marginals_maj = majority_vote(L_unlab)

    
    # Discriminative model learning
    ## Without GS

    plot_marginals_histogram(unlab_marginals_gen, 
                             title = 'Histogram of marginals' ,
                            bins = 20)

    train_kwargs = {
        'lr':         0.01,
        'dim':        100,
        'n_epochs':   25,
        'dropout':    0.5,
        'rebalance':  True,
        'print_freq': 1,
        'batch_size': 64
    }

    lstm = reRNN(seed=1701, n_threads=6,)
    lstm.train(unlab, unlab_marginals_gen , X_dev=dev, Y_dev=L_gold_dev, dev_ckpt_delay=0, **train_kwargs)

    test_scores.loc[result_dict_file, 'LSTM + GM_P'], test_scores.loc[result_dict_file, 'LSTM + GM_R'], test_scores.loc[result_dict_file, 'LSTM + GM_F1'] = lstm.score(test, L_gold_test)

    #** with averaging marginals **

    train_kwargs = {
        'lr':         0.01,
        'dim':        100,
        'n_epochs':   25,
        'dropout':    0.5,
        'rebalance':  True,
        'print_freq': 1,
        'batch_size': 64
    }

    lstm = reRNN(seed=1701, n_threads=6,)
    lstm.train(unlab, unlab_marginals_avg , X_dev=dev, Y_dev=L_gold_dev,  dev_ckpt_delay=0, **train_kwargs)

    test_scores.loc[result_dict_file, 'LSTM + AvgV_P'], test_scores.loc[result_dict_file, 'LSTM + AvgV_R'], test_scores.loc[result_dict_file, 'LSTM + AvgV F1'] = lstm.score(test, L_gold_test)


    train_kwargs = {
        'lr':         0.01,
        'dim':        100,
        'n_epochs':   25,
        'dropout':    0.5,
        'rebalance':  True,
        'print_freq': 1,
        'batch_size': 64
    }

    lstm = reRNN(seed=1701, n_threads=6,)
    lstm.train(unlab, unlab_marginals_maj , X_dev=dev, Y_dev=L_gold_dev, dev_ckpt_delay=0, **train_kwargs)

    test_scores.loc[result_dict_file, 'LSTM + MV_P'], test_scores.loc[result_dict_file, 'LSTM + MV_R'], test_scores.loc[result_dict_file, 'LSTM + MV_F1'] = lstm.score(test, L_gold_test)

Loaded:


['results_dict,lemmas,RuS,CV_,bin_,minFreq=5,_ngrams=(0, 3)_SVC_rbf_C=75',
 'results_dict,ShortDepPath,lemmas,RuS,TfIdf_,minFreq=5,_ngrams=(0, 3),LSA200_SVC_linear',
 'results_dict,trim=5,lemmas,RuS,TfIdf_,minFreq=5,_ngrams=(0, 3)_SVC_linear',
 'results_dict,trim=5,lemmas,RuS,TfIdf_,minFreq=5,_stopw=english,_ngrams=(1, 1),LSA200_SVC_linear',
 'results_dict,trim=5,lemmas,RuS,TfIdf_,minFreq=5,_ngrams=(0, 3)_RandomForestClassifier',
 'biLSTM_biLSTM_lr=0.001',
 'results_dict,trim=0,lemmas,RuS,TfIdf_,minFreq=5,_ngrams=(0, 3),LSA200_SVC_linear',
 'results_dict,lemmas,RuS,TfIdf_,minFreq=5,_stopw=english,_ngrams=(1, 1),LSA200_SVC_rbf_C=75',
 'results_dict,ShortDepPath,lemmas,RuS,CV_,bin_,minFreq=5,_stopw=english,_ngrams=(1, 1),LSA200_LogisticRegression',
 'results_dict,trim=0,lemmas,RuS,TfIdf_,minFreq=5,_stopw=english,_ngrams=(1, 1),LSA200_SVC_rbf_C=250']

## With GS

In [89]:
# from MLutils import balance_candidates

In [90]:
# from snorkel.annotations import load_marginals
# train_labels = np.array([0 if train[x].gold_labels[0].value ==-1 else 1 for x in range(len(train))]) # load train labels to mix in LSTM
# unlab_marginals = load_marginals(session, split=3)


In [91]:
# #manually balance candidates
# train_us, train_labels_us = balance_candidates(train,train_labels)
# unlab_us, unlab_marginals_us = balance_candidates(unlab,unlab_marginals)

In [92]:
# train_unlab_marginals = np.append(train_labels_us, unlab_marginals_us)

In [93]:
# plt.hist(train_unlab_marginals, bins=20)
# plt.title('Histogram of marginals (candidates undersampled before GM) + GS ')
# plt.show()

In [94]:
# train_kwargs = {
#     'lr':         0.01,
#     'dim':        100,
#     'n_epochs':   25,
#     'dropout':    0.5,
#     'rebalance':  False,
#     'print_freq': 1,
#     'batch_size': 64
# }

# lstm = reRNN(seed=1701, n_threads=6)
# lstm.train(train_us+unlab_us, train_unlab_marginals, X_dev=dev, Y_dev=L_gold_dev, dev_ckpt_delay=0, **train_kwargs)


In [95]:
# %%capture cap --no-stderr
# tp, fp, tn, fn = lstm.error_analysis(session, dev, L_gold_dev, batch_size = 1024)

In [96]:
# %%capture cap --no-stderr
# with open(result_dict_file+".txt", "a") as f:
#     f.write(unicode("LSTM + GM (val set) trained on GS+US:\n"))
#     f.write(cap.stdout)
#     f.write(unicode('\n\n'))

In [97]:
# %%capture cap --no-stderr
# tp, fp, tn, fn = lstm.error_analysis(session, test, L_gold_test, batch_size = 1024)

In [98]:
# with open(result_dict_file+".txt", "a") as f:
#     f.write(unicode("LSTM + GM (test set) trained on GS+US:\n"))
#     f.write(cap.stdout)
#     f.write(unicode('\n\n'))

In [99]:
# test_scores.loc[result_dict_file, 'LSTM + GM (GS+US)'] = lstm.score(test, L_gold_test)[-1]

### Save scores

In [86]:
with open(result_dict_file+".txt", "a") as f:
    f.write(unicode(test_scores.to_latex()))
    f.write(unicode('\n\n'))
    

AttributeError: 'newlist' object has no attribute 'to_latex'